# DSPy RAG Experiment

Dieses Notebook zeigt DSPy-Varianten fuer RAG (Baseline, Few-shot, MIPRO)
inkl. leichtgewichtigem Modell-Setup. Es baut auf den Retrieval-Kontexten
auf und testet die Optimierung von Prompt/Programmen.

**Hinweise:**
- Passen Sie `model` und `top_k` je nach Ziel an.
- Fuer schnelle Runs kann ein kleineres Modell verwendet werden.


## Dataset erstellen (CSV + Qdrant)

Lädt Fragen/Antworten aus der CSV, holt Kontexte aus Qdrant und erzeugt ein `dataset`.


In [ ]:
import pandas as pd
from pathlib import Path
from datasets import Dataset
from litellm_client import (
    load_llm_config,
    load_vectordb_config,
    get_qdrant_client,
    get_embeddings,
)

def _retrieve_contexts(question: str, k: int, client, collection_name: str, llm_cfg):
    query_emb = get_embeddings([question], llm_cfg, batch_size=1)[0]
    results = client.query_points(
        collection_name=collection_name,
        query=query_emb,
        limit=k,
    ).points
    return [res.payload.get('text', '') for res in results]

def build_eval_dataset(
    csv_path: str = '../data/data_evaluation/GSKI_Fragen-Antworten-Fundstellen.csv',
    top_k: int = 5,
) -> Dataset:
    llm_cfg = load_llm_config()
    vec_cfg = load_vectordb_config()
    qdrant_client = get_qdrant_client(vec_cfg)
    collection_name = vec_cfg.collection or 'grundschutz_xml'

    df = pd.read_csv(Path(csv_path), sep=';', encoding='utf-8-sig')
    records = []

    for _, row in df.iterrows():
        question = row['Frage']
        ground_truth_answer = row['Antwort']
        ground_truth_context = row['Fundstellen im IT-Grundschutz-Kompendium 2023']

        contexts = _retrieve_contexts(question, top_k, qdrant_client, collection_name, llm_cfg)

        records.append({
            'question': question,
            'contexts': contexts,
            'ground_truth_answer': ground_truth_answer,
            'ground_truth_context': ground_truth_context,
        })

    return Dataset.from_list(records)

dataset = build_eval_dataset(top_k=3)
dataset


/Users/felixboelter/Documents/GitHub/pilotprojekt-GrundschutzKI/notebooks/litellm_client.py:177: UserWarning: Api key is used with an insecure connection.
  return QdrantClient(url=url, api_key=cfg.api_key)


Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing

Dataset({
    features: ['question', 'contexts', 'ground_truth_answer', 'ground_truth_context'],
    num_rows: 42
})

In [17]:
import dspy
from litellm_client import load_llm_config

llm_cfg = load_llm_config()
model = "openai/granite-4-h-tiny"

# LiteLLM‑Proxy (OpenAI‑kompatibel)
dspy_llm = dspy.LM(
    model=model,
    api_base=llm_cfg.api_base,
    api_key=llm_cfg.api_key,
)

dspy.configure(lm=dspy_llm)


In [18]:
def format_docs(docs):
    # docs = list of strings
    doc_list = [
        {"doc_id": i+1, "title": "", "text": d, "source": ""}
        for i, d in enumerate(docs)
    ]
    return doc_list

class RAGAnswer(dspy.Signature):
    """Antworte auf Deutsch, kurz und präzise, max. 2–3 Sätze.
    Nutze ausschließlich die bereitgestellten Dokumente.
    Wenn die Antwort nicht in den Dokumenten steht, sage das."""
    question: str = dspy.InputField()
    response: str = dspy.OutputField(desc="Antwort auf Deutsch, kurz und präzise, maximal 2–3 Sätze.")

class RAGModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(
            RAGAnswer,
        )

    def forward(self, question, contexts):
        return self.predict(
            question=question,
            config={"chat_template_kwargs": {"documents": format_docs(contexts)}},
        )


rag = RAGModule()


In [19]:
# Beispielausgaben
for i in range(3):
    row = dataset[i]
    contexts = row["contexts"]
    pred = rag(question=row["question"], contexts=contexts)
    print(f"\n--- SAMPLE {i} ---")
    print("QUESTION:", row["question"])
    print("PREDICTED ANSWER:", pred.response)
    print("GROUND TRUTH:", row["ground_truth_answer"])



--- SAMPLE 0 ---
QUESTION: Was ist der Unterschied zwischen Prozess- und Systembausteinen?
PREDICTED ANSWER: Prozess-Bausteine gelten in der Regel für sämtliche oder große Teile des Informationsverbunds gleichermaßen, während System-Bausteine in der Regel auf einzelne Objekte oder Gruppen von Objekten anzuwenden sind. Beide bestehen wiederum aus weiteren Teilschichten und werden basierend auf ihrer Relevanz und Priorität zur Umsetzung im IT-Grundschutz ausgewählt. Prozess-Bausteine sind z.B. das Sicherheitsmanagement (ISMS) und der Datenschutz (CON.2), während System-Bausteine wie der Anforderungen für clientseitige Betriebssysteme (APP.2.1) und das Netzwerkinfrastrukturschutz (APP.3.1) Beispiel dafür sind. Die Auswahl der Bausteine erfolgt in einer bestimmten Reihenfolge und richtet sich sowohl an Prozess- als auch an System-Bausteine.
GROUND TRUTH: Prozess-Bausteine gelten in der Regel für sämtliche oder große Teile des Informationsverbunds gleichermaßen, System-Bausteine lassen sic

## DSPy Optimizer (MIPROv2)

Optimiert die Prompt‑Instruktionen für das RAG‑Programm. Kann kostenintensiv sein.


In [20]:
import nest_asyncio, asyncio
from ragas.metrics.collections import AnswerCorrectness
from ragas.embeddings.litellm_provider import LiteLLMEmbeddings
from ragas.llms import llm_factory
import instructor, litellm
nest_asyncio.apply()
# RAGAS LLM
litellm.api_base = llm_cfg.api_base
litellm.api_key = llm_cfg.api_key
client = instructor.from_litellm(litellm.acompletion, mode=instructor.Mode.MD_JSON)
ragas_llm = llm_factory(llm_cfg.model, client=client, adapter="litellm")

# Embeddings (für Similarity)
embeddings = LiteLLMEmbeddings(
    model=llm_cfg.embedding_model,
    api_key=llm_cfg.api_key,
    api_base=llm_cfg.api_base,
    encoding_format="float",
)

ac = AnswerCorrectness(llm=ragas_llm, embeddings=embeddings)

def ragas_ac_metric(example, pred, trace=None):
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(
        ac.ascore(
            user_input=example.question,
            response=pred.response,
            reference=example.response,
        )
    )
    return result.value




In [21]:
import dspy
from dspy.evaluate import SemanticF1

# DSPy Examples aus dem vorhandenen Dataset
examples = []
for row in dataset:
    examples.append(
        dspy.Example(question=row["question"], contexts=row["contexts"], response=row["ground_truth_answer"])
            .with_inputs("question", "contexts")
    )


# einfache Splits
trainset = examples[: max(1, len(examples)//5)]
devset = examples[max(1, len(examples)//5):]

metric = SemanticF1(decompositional=False)

# Optimizer (wenig Threads zum Start)
tp = dspy.MIPROv2(metric=metric, auto='light', num_threads=6)
optimized_rag = tp.compile(rag, trainset=trainset)


2026/02/03 10:20:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 6

2026/02/03 10:20:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2026/02/03 10:20:40 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2026/02/03 10:20:40 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


100%|██████████| 2/2 [00:23<00:00, 11.55s/it]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/6


100%|██████████| 2/2 [00:00<00:00, 14.58it/s]


...


Average Metric: 3.09 / 6 (51.5%): 100%|██████████| 6/6 [00:00<00:00, 2771.57it/s]

2026/02/03 10:23:13 INFO dspy.evaluate.evaluate: Average Metric: 3.0901360544217686 / 6 (51.5%)
2026/02/03 10:23:13 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 51.5 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 5'].
2026/02/03 10:23:13 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.04, 87.32, 79.78, 80.18, 71.17, 87.81, 79.78, 51.5, 80.18, 51.5]
2026/02/03 10:23:13 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 87.81
2026/02/03 10:23:13 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2026/02/03 10:23:13 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 11 / 10 =====



Average Metric: 5.27 / 6 (87.8%): 100%|██████████| 6/6 [00:00<00:00, 914.22it/s]

2026/02/03 10:23:14 INFO dspy.evaluate.evaluate: Average Metric: 5.268375230473298 / 6 (87.8%)
2026/02/03 10:23:14 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 87.81 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 5'].
2026/02/03 10:23:14 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.04, 87.32, 79.78, 80.18, 71.17, 87.81, 79.78, 51.5, 80.18, 51.5, 87.81]
2026/02/03 10:23:14 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 87.81


2026/02/03 10:23:14 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2026/02/03 10:23:14 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 87.81!


In [22]:
split = max(1, len(dataset) // 5)
dev_rows = list(dataset)[split:]
dspy_dev_answers = []
for i, row in enumerate(dev_rows):
    print(f"\n--- SAMPLE {i+1} ---")
    contexts = row["contexts"]
    pred = rag(question=row["question"], contexts=contexts)
    dspy_dev_answers.append(pred.response)
    print("QUESTION:", row["question"])
    print("PREDICTED ANSWER:", pred.response)
    print("GROUND TRUTH:", row["ground_truth_answer"])

# Neues Dataset fürs Scoring
from datasets import Dataset

dev_dataset = Dataset.from_dict({
    "question": [r["question"] for r in dev_rows],
    "contexts": [r["contexts"] for r in dev_rows],
    "ground_truth_answer": [r["ground_truth_answer"] for r in dev_rows],
    "ground_truth_context": [r["ground_truth_context"] for r in dev_rows],
    "answer": dspy_dev_answers,   # <- DSPy Antworten
})



--- SAMPLE 1 ---
QUESTION: Was ist bei der Auswahl eines externen Webhosters zu beachten?
PREDICTED ANSWER: Bei der Auswahl eines externen Webhosters SOLLTE vertraglich geregelt werden, wie die Dienste zu erbringen sind. Es SOLLTE vertraglich geregelt werden, wie die Dienste zu erbringen sind. Dabei SOLLTEN Sicherheitsaspekte innerhalb des Vertrags schriftlich in einem Service Level Agreement (SLA) festgehalten werden. Die eingesetzten IT-Systeme SOLLTEN vom Webhoster regelmäßig kontrolliert und gewartet werden. Der Webhoster SOLLTE dazu verpflichtet werden, bei technischen Problemen oder einer Kompromittierung von Kundensystemen zeitnah zu reagieren. Der Webhoster SOLLTE grundlegende technische und organisatorische Maßnahmen umsetzen, um seinen Informationsverbund zu schützen. Aus den HTTP-Informationen und den angezeigten Fehlermeldungen SOLLTEN weder der Produktname noch die verwendete Version des Webservers ersichtlich sein. Fehlermeldungen SOLLTEN keine Details zu Systeminformati

In [23]:
optimized_rag

predict = Predict(StringSignature(question -> response
    instructions='Antworte auf Deutsch, kurz und präzise, max. 2–3 Sätze.\nNutze ausschließlich die bereitgestellten Dokumente.\nWenn die Antwort nicht in den Dokumenten steht, sage das.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    response = Field(annotation=str required=True json_schema_extra={'desc': 'Antwort auf Deutsch, kurz und präzise, maximal 2–3 Sätze.', '__dspy_field_type': 'output', 'prefix': 'Response:'})
))

## RAGAS Evaluation (DSPy Answers)


In [ ]:
import asyncio
from ragas.llms import llm_factory
from ragas.embeddings.litellm_provider import LiteLLMEmbeddings
from ragas.metrics.collections import ContextPrecision, ContextRecall, Faithfulness, AnswerCorrectness, NoiseSensitivity
import instructor
import litellm

# RAGAS LLM (LiteLLM proxy)
litellm.api_base = llm_cfg.api_base
litellm.api_key = llm_cfg.api_key
client = instructor.from_litellm(litellm.acompletion, mode=instructor.Mode.MD_JSON)
ragas_llm = llm_factory(llm_cfg.model, client=client, adapter='litellm', model_args={'temperature': 0.2})

embeddings = LiteLLMEmbeddings(
    model=llm_cfg.embedding_model,
    api_key=llm_cfg.api_key,
    api_base=llm_cfg.api_base,
    encoding_format='float',
)

scorers = {
    "context_precision": ContextPrecision(llm=ragas_llm),
    "context_recall": ContextRecall(llm=ragas_llm),
    "faithfulness": Faithfulness(llm=ragas_llm),
    "answer_correctness": AnswerCorrectness(llm=ragas_llm, embeddings=embeddings),
    "noise_sensitivity": NoiseSensitivity(llm=ragas_llm),
    "noise_sensitivity_irrelevant": NoiseSensitivity(llm=ragas_llm, mode="irrelevant"),
}

async def _score_row(row, sem):
    async with sem:
        return {
            "context_precision": (await scorers["context_precision"].ascore(
                user_input=row["question"],
                reference=row["ground_truth_context"],
                retrieved_contexts=row["contexts"],
            )).value,
            "context_recall": (await scorers["context_recall"].ascore(
                user_input=row["question"],
                reference=row["ground_truth_context"],
                retrieved_contexts=row["contexts"],
            )).value,
            "faithfulness": (await scorers["faithfulness"].ascore(
                user_input=row["question"],
                response=row["answer"],
                retrieved_contexts=row["contexts"],
            )).value,
            "answer_correctness": (await scorers["answer_correctness"].ascore(
                user_input=row["question"],
                response=row["answer"],
                reference=row["ground_truth_answer"],
            )).value,
            "noise_sensitivity": (await scorers["noise_sensitivity"].ascore(
                user_input=row["question"],
                response=row["answer"],
                reference=row["ground_truth_answer"],
                retrieved_contexts=row["contexts"],
            )).value,
            "noise_sensitivity_irrelevant": (await scorers["noise_sensitivity_irrelevant"].ascore(
                user_input=row["question"],
                response=row["answer"],
                reference=row["ground_truth_answer"],
                retrieved_contexts=row["contexts"],
            )).value,
        }


async def score_dataset_batched(ds, batch_size=10, concurrency=5):
    sem = asyncio.Semaphore(concurrency)
    rows = list(ds)
    results = []
    for i in range(0, len(rows), batch_size):
        batch = rows[i : i + batch_size]
        tasks = [asyncio.create_task(_score_row(r, sem)) for r in batch]
        results.extend(await asyncio.gather(*tasks))
    return results

scores = await score_dataset_batched(dev_dataset, batch_size=16, concurrency=10)
stats = {
    k: {
        'avg': sum(s[k] for s in scores) / len(scores),
        'min': min(s[k] for s in scores),
        'max': max(s[k] for s in scores),
    }
    for k in scores[0].keys()
}
print(stats)


{'context_precision': {'avg': 0.9027941176200859, 'min': 0.4166666666458333, 'max': 0.9999999999833333}, 'context_recall': {'avg': 0.8255832546922512, 'min': 0.09090909090909091, 'max': 1.0}, 'faithfulness': {'avg': 0.7559134814644196, 'min': 0.0, 'max': 1.0}, 'answer_correctness': {'avg': 0.5016627205310639, 'min': 0.1328630119917154, 'max': 0.9818494666870119}}


## Ergebnis-DataFrame


In [ ]:
import pandas as pd

# DSPy-Antworten erzeugen (optimized_rag falls vorhanden)
_rag_model = optimized_rag if 'optimized_rag' in globals() else rag
dspy_answers = []
for row in dataset:
    contexts = row['contexts']
    pred = _rag_model(question=row['question'], contexts=contexts)
    dspy_answers.append(pred.response)


# DataFrame zusammenbauen
df = pd.DataFrame({
    'question': [r['question'] for r in dev_dataset],
    'contexts': ["\n\n".join(r['contexts']) for r in dev_dataset],
    'answer_dspy': dspy_dev_answers,
    'ground_truth': [r['ground_truth_answer'] for r in dev_dataset],
    'context_precision': [s['context_precision'] for s in scores],
    'context_recall': [s['context_recall'] for s in scores],
    'faithfulness': [s['faithfulness'] for s in scores],
    'answer_correctness': [s['answer_correctness'] for s in scores],
})


df.head()


,question,contexts,answer_dspy,ground_truth,context_precision,context_recall,faithfulness,answer_correctness
0,Was ist bei der Auswahl eines externen Webhost...,"n besorgt werden. Auch SOLLTE geregelt werden,...",Bei der Auswahl eines externen Webhosters soll...,Bei der Nutzung externer Webhosting-Dienste so...,1.000000,1.000000,1.000,0.584186
1,Wie sollten Fehlermeldungen auf einem Webserve...,"n besorgt werden. Auch SOLLTE geregelt werden,...",Fehl\nmeldungen sollten weder den Produktnamen...,Aus HTTP-Antworten und Fehlermeldungen dürfen ...,1.000000,1.000000,0.625,0.900584
2,Welche Maßnahmen sind bei erhöhtem Schutzbedar...,ei erhöhtem Schutzbedarf\nIm Folgenden sind fü...,Maßnahmen bei erhöhtem Schutzbedarf für Webser...,Bei erhöhtem Schutzbedarf sollten Webserver re...,0.333333,0.928571,0.500,0.214799
3,Wer trägt die Verantwortung für Informationssi...,nehmen (Anbietende von Outsourcing) aus.\n\nDi...,Der oder die Informationssicherheitsbeauftragt...,Die Verantwortung für Informationssicherheit v...,1.000000,0.600000,1.000,0.463193
4,Welche Prozesse dürfen grundsätzlich ausgelage...,in eckigen Klammern aufgeführt. Die Verwendung...,Grundsätzlich können Prozesse ausgelagert werd...,"Nur Prozesse, die risikoorientiert bewertet wu...",1.000000,1.000000,0.750,0.495391


In [ ]:
print(df["question"][10])
print()
print(df["answer_dspy"][10])
print()
print(df["ground_truth"][10])


In [ ]:
print(df["contexts"][10])

## Findings (Top‑K = 3)

### 1) Retrieval
- **Context Precision:** 0.95 (hoch; wenig irrelevanter Kontext)  
- **Context Recall:** 0.82 (gut, aber nicht vollständig)

### 2) Antwortqualität
- **Faithfulness:** 0.76 (moderat; teils nicht vollständig kontextgebunden)  
- **Answer Correctness:** 0.53 (niedrig, aber stark vom Datensatz beeinflusst)

### 3) Datensatz‑Einfluss
- **GT‑Faithfulness:** ~0.83  
- **GT‑Relevancy:** ~0.63  
→ Korrigiert die Obergrenze für Answer Correctness nach unten. Ein Teil der niedrigen Werte ist daher **Dataset‑Limit**, nicht zwingend Modellfehler.

### 4) Robustheit ggü. Noise
- **Noise Sensitivity:** 0.29 (moderat)  
- **Noise Sensitivity (irrelevant):** 0.11 (relativ robust)

### 5) Nächste Schritte
- Recall verbessern (z. B. höheres `top_k` / besseres Chunking).  
- Antwortstil stärker extraktiv machen, um Faithfulness/Korrektheit zu erhöhen.
